# Corretor Ortográfico Python

## Criando base de treinamento para o corretor ortográfico

### Incluindo corpus textual

In [1]:
with open("dados/artigos.txt","r", encoding='utf8') as f:
    artigos = f.read()
print(artigos[:300])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação


In [2]:
len(artigos)

2605046

### Criando tokens

In [3]:
texto_exemplo = 'Olá, tudo bem?'
tokens = texto_exemplo.split()
tokens

['Olá,', 'tudo', 'bem?']

In [4]:
len(tokens)

3

### Criando tokens contendo apenas palavras

Para isso será utilizada a biblioteca Natural Language Toolkit (nltk)

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafa_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)

In [7]:
palavras_separadas

['Olá', ',', 'tudo', 'bem', '?']

In [8]:
len(palavras_separadas)

5

In [9]:
'palavra'.isalpha()

True

In [10]:
'...!'.isalpha()

False

In [11]:
def retira_pontuacao(lista_tokens):
    lista_palavras = []
    
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
            
    return lista_palavras

In [12]:
retira_pontuacao(palavras_separadas)

['Olá', 'tudo', 'bem']

### Aplicando técnica de 'tokenização' no corpus textual

In [13]:
tokens = nltk.tokenize.word_tokenize(artigos)

In [14]:
len(tokens)

515905

In [15]:
lista_palavras = retira_pontuacao(tokens)

In [16]:
len(lista_palavras)

403104

In [17]:
lista_palavras[:5]

['imagem', 'Temos', 'a', 'seguinte', 'classe']

### Normalizando tokens para que contenham apenas caracteres minúsculos

In [18]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
        
    return lista_normalizada

In [19]:
lista_normalizada = normalizacao(lista_palavras)

In [20]:
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

### Removendo palavras repetidas

In [21]:
set([1,2,2,3,3,3,4,4,5,6])

{1, 2, 3, 4, 5, 6}

In [22]:
len(set(lista_normalizada))

18465

In [23]:
len(lista_normalizada)

403104

### Algoritmo de correção ortográfica

#### O algoritmo analisará toda a palavra em busca de caractere faltante, fatiando a palavra em parte Esquerda e parte Direita, incluindo entre as duas partes possíveis combinações de letras, gerando resultados diversos que serão comparados com o banco ,corpus textual, e retornará a palavra corrigida.

In [24]:
#fatiando listas
lista = [1,2,3]
lista[1:]

[2, 3]

In [25]:
#fatiando string
lista = 'lgica'
lista[1:3]

'gi'

In [26]:
#fatiando string em lado esquerdo e direito
(lista[:2],lista[2:])

('lg', 'ica')

In [27]:
#função que gera novas palavras (v.1)
palavra_exemplo = 'lgica'
def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra)):
        fatias.append((palavra[:i],palavra[i:]))
    print(fatias)
    
    #palavras_geradas = insere_letras(fatias)
    #return palavras_geradas
gerador_palavras(palavra_exemplo)

[('', 'lgica'), ('l', 'gica'), ('lg', 'ica'), ('lgi', 'ca'), ('lgic', 'a')]


In [28]:
#em v.1 a função não gera o último caso previsto --> ('lgica','')
#para resolver, o for será reescrito --> for i in range(len(palavra) + 1)

#função que gera novas palavras (v.2)
palavra_exemplo = 'lgica'
def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
    print(fatias)
    
gerador_palavras(palavra_exemplo)

[('', 'lgica'), ('l', 'gica'), ('lg', 'ica'), ('lgi', 'ca'), ('lgic', 'a'), ('lgica', '')]


#### Algorítmo de geração de possíveis palavras

In [29]:
palavra_exemplo = 'lgica'

def insere_letras(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
        
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas
    
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas[:10])

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica']


#### Algorítmo do corretor

In [30]:
total_palavras = len(lista_normalizada)
frequencia = nltk.FreqDist(lista_normalizada)
print(frequencia.most_common(10))
print(frequencia['lógica'])

[('de', 15502), ('o', 14056), ('que', 12230), ('a', 11099), ('e', 10501), ('para', 7710), ('um', 6368), ('é', 5899), ('uma', 5220), ('do', 5124)]
96


In [31]:
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

probabilidade('lógica')

0.00023815194093831864

In [32]:
max(palavras_geradas, key=probabilidade)

'lógica'

In [33]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [34]:
corretor(palavra_exemplo)

'lógica'

In [35]:
corretor('clculo')

'cálculo'

## Agrupando algorítmo de correção de palavra para o caso de falta de um caractere. 'lgica' -> 'lógica'

In [36]:
total_palavras = len(lista_normalizada)
frequencia = nltk.FreqDist(lista_normalizada)
palavra_exemplo = 'lgica'

def insere_letras(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
        
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

### Avaliador da taxa de acerto do corretor

In [37]:
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding='utf8')
   
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste



In [38]:
lista_teste = cria_dados_teste('Dados/palavras.txt')
lista_teste[:10]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm')]

In [39]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertos = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertos += 1
    taxa_acerto = round(acertos*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras')

In [40]:
avaliador(lista_teste)

1.08% de 186 palavras


## Melhorando o corretor (parte 1) -  Correção de palavras com letra a mais. 'lóigica' -> 'lógica'

In [41]:
'palavra'[1:]

'alavra'

In [42]:
def deleta_letra(fatias):
    novas_palavras = []
    
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

In [43]:
def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letra(fatias)
    return palavras_geradas

In [44]:
palavra_exemplo = 'lóigica'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas[:10])

['alóigica', 'blóigica', 'clóigica', 'dlóigica', 'elóigica', 'flóigica', 'glóigica', 'hlóigica', 'ilóigica', 'jlóigica']


In [45]:
avaliador(lista_teste)

41.4% de 186 palavras


## Melhorando o corretor (parte 2) - Correção de troca de letras. 'lígica' -> 'lógica'

In [46]:
def insere_letras(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def deleta_letra(fatias):
    novas_palavras = []
    
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

def troca_letra(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras    

def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letra(fatias)
    palavras_geradas += troca_letra(fatias)
    return palavras_geradas

palavra_exemplo = 'lígica'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas[:10])

['alígica', 'blígica', 'clígica', 'dlígica', 'elígica', 'flígica', 'glígica', 'hlígica', 'ilígica', 'jlígica']


In [47]:
avaliador(lista_teste)

76.34% de 186 palavras


## Melhorando o corretor (parte 3) - Correção de letras em posições trocadas. 'lgóica' -> 'lógica'

In [48]:
def insere_letras(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def deleta_letra(fatias):
    novas_palavras = []
    
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

def troca_letra(fatias):
    novas_palavras = []
    alfabeto = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for E, D in fatias:
        for letra in alfabeto:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras    

def inverte_letras(fatias):
    novas_palavras = []
    
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letra(fatias)
    palavras_geradas += troca_letra(fatias)
    palavras_geradas += inverte_letras(fatias)
    return palavras_geradas

palavra_exemplo = 'lógiac'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas[:10])

['alógiac', 'blógiac', 'clógiac', 'dlógiac', 'elógiac', 'flógiac', 'glógiac', 'hlógiac', 'ilógiac', 'jlógiac']


In [49]:
avaliador(lista_teste)

76.34% de 186 palavras


In [50]:
corretor('lógiac')

'lógica'

In [51]:
corretor('lógicaa')

'lógica'

## Avaliando quanto do erro está relacionado ao tamanho do vocabulário da função avaliador()

In [52]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertos = 0
    desconhecida = 0
    
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertos += 1
        else:
            desconhecida += (correta not in vocabulario)
    taxa_acerto = round(acertos*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'Acertou {taxa_acerto}% de {numero_palavras} palavras. Desconhecidas são {taxa_desconhecida}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)
print(f'Palavras no vocalublário: {len(vocabulario)}')

Acertou 76.34% de 186 palavras. Desconhecidas são 6.99%
Palavras no vocalublário: 18465


## Melhorando o corretor (parte 4) - Correção de palavras com até dois erros. 'lóiigica' -> 'lógica'

In [53]:
palavra = 'lóiigica'

def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
           
    return novas_palavras

In [54]:
palavras_g = gerador_turbinado(gerador_palavras(palavra))
"lógica" in palavras_g

True

In [55]:
len(palavras_g)

691744

### Filtrando a quantidade de palavras geradas, e assim otimizar o processo de cálculos de probabilidade, uma vez que para uma única palavra são geradas mais de meio milhão de possibilidades.

In [56]:
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_turbinada = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavra_turbinada)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
            candidatos.append(palavra) 
                       
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

In [57]:
novo_corretor('líguaa')

'língua'

### Reescrevendo a função avaliador() para utilizar a função novo_corretor()

Para o novo_corretor

In [58]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertos = 0
    desconhecida = 0
    
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertos += 1
        else:
            pass
           # print(errada + '-' + corretor(errada) + '-' + palavra_corrigida)
    taxa_acerto = round(acertos*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras, desconhecida é {taxa_desconhecida}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

55.38% de 186 palavras, desconhecida é 6.99%


Para corretor

In [59]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertos = 0
    desconhecida = 0
    
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertos += 1
        
    taxa_acerto = round(acertos*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras, desconhecida é {taxa_desconhecida}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras, desconhecida é 6.99%


Correção com os dois corretores criados

In [60]:
palavra = 'lóogica'
print(corretor(palavra))
print(novo_corretor(palavra))

lógica
lógica
